In [3]:
%use ktor-client
%use coroutines
%use serialization

import io.ktor.client.*
import io.ktor.client.call.*
import io.ktor.client.engine.cio.*
import io.ktor.client.plugins.contentnegotiation.*
import io.ktor.client.request.*
import io.ktor.serialization.kotlinx.json.*
import kotlinx.serialization.json.*
import kotlinx.coroutines.*


val repo = "Erick194/DoomRPG-RE"

runBlocking {
    // Создаем клиент и сразу настраиваем JSON
    val client = HttpClient(CIO) {
        install(ContentNegotiation) {
            // Игнорируем лишние поля, чтобы не было ошибок
            json(Json { ignoreUnknownKeys = true })
        }
    }

    // Используем .use, чтобы клиент корректно закрылся после работы
    client.use {
        try {
            val response = it.get("https://api.github.com/repos/$repo/releases/latest") {
                // GitHub API обязательно требует User-Agent
                header("User-Agent", "Kotlin-Notebook")
            }.body<JsonObject>()

            // Пробираемся через JSON: assets -> [0] -> browser_download_url
            val downloadUrl = response["assets"]
                ?.jsonArray
                ?.firstOrNull()
                ?.jsonObject
                ?.get("browser_download_url")
                ?.jsonPrimitive
                ?.content

            if (downloadUrl != null) {
                println("✅ Прямая ссылка получена:\n$downloadUrl")
            } else {
                println("❌ Ссылка не найдена (возможно, в релизе нет прикрепленных файлов).")
            }
        } catch (exception: Exception) {
            println("❌ Ошибка запроса: ${exception.message}")
        }
    }
}

https://github.com/Erick194/DoomRPG-RE/releases/download/v.0.2.2/DoomRPG_Port.-.v.0.2.2.zip
